# Exploration

## Try HuggingFace Model

In [2]:
from transformers import pipeline

# Load a semantic role labeling pipeline
srl_pipeline = pipeline("token-classification", model="liaad/srl-pt_bertimbau-large")

# Apply SRL
text = "Tesla unveiled its new electric truck in California last night."
results = srl_pipeline(text)

# Print extracted roles
for entity in results:
    print(entity)


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at liaad/srl-pt_bertimbau-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entity': 'LABEL_1', 'score': 0.50187516, 'index': 1, 'word': 'Tes', 'start': 0, 'end': 3}
{'entity': 'LABEL_0', 'score': 0.5490839, 'index': 2, 'word': '##la', 'start': 3, 'end': 5}
{'entity': 'LABEL_0', 'score': 0.5232201, 'index': 3, 'word': 'un', 'start': 6, 'end': 8}
{'entity': 'LABEL_0', 'score': 0.54658175, 'index': 4, 'word': '##ve', 'start': 8, 'end': 10}
{'entity': 'LABEL_0', 'score': 0.507578, 'index': 5, 'word': '##ile', 'start': 10, 'end': 13}
{'entity': 'LABEL_0', 'score': 0.5712971, 'index': 6, 'word': '##d', 'start': 13, 'end': 14}
{'entity': 'LABEL_0', 'score': 0.52105176, 'index': 7, 'word': 'it', 'start': 15, 'end': 17}
{'entity': 'LABEL_0', 'score': 0.5483713, 'index': 8, 'word': '##s', 'start': 17, 'end': 18}
{'entity': 'LABEL_0', 'score': 0.52619505, 'index': 9, 'word': 'ne', 'start': 19, 'end': 21}
{'entity': 'LABEL_0', 'score': 0.53404623, 'index': 10, 'word': '##w', 'start': 21, 'end': 22}
{'entity': 'LABEL_1', 'score': 0.5083904, 'index': 11, 'word': 'electr'

## With AllenNLP

In [34]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from allennlp.data.tokenizers import SpacyTokenizer

# Load the SRL model
predictor = Predictor.from_path(
    "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"
)

# Manually set tokenizer
predictor._tokenizer = SpacyTokenizer(language="en_core_web_sm", pos_tags=True)

# Test sentence
sentence = "Protesters blocked the main highway in Paris last night due to rising fuel prices."

# Predict SRL roles
output = predictor.predict(sentence)
print(output)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'blocked', 'description': '[ARG0: Protesters] [V: blocked] [ARG1: the main highway in Paris] [ARGM-TMP: last night] [ARGM-CAU: due to rising fuel prices] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'B-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'O']}, {'verb': 'rising', 'description': 'Protesters blocked the main highway in Paris last night due to [V: rising] [ARG1: fuel prices] .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['Protesters', 'blocked', 'the', 'main', 'highway', 'in', 'Paris', 'last', 'night', 'due', 'to', 'rising', 'fuel', 'prices', '.']}


In [6]:
def combine_bio_tags(output):
    verbs_data = output['verbs']
    words = output['words']
    event_representation = []

    for verb_data in verbs_data:
        verb = verb_data["verb"]
        description = verb_data["description"]
        tags = verb_data["tags"]

        combined_roles = {}
        current_role = None
        current_phrase = []

        for word, tag in zip(words, tags):
            if tag.startswith("B-"):  # Beginning of a new entity
                if current_role:
                    combined_roles[current_role] = " ".join(current_phrase)
                current_role = tag[2:]  # Remove "B-"
                current_phrase = [word]
            elif tag.startswith("I-") and current_role:  # Continuation of entity
                current_phrase.append(word)
            else:
                if current_role:
                    combined_roles[current_role] = " ".join(current_phrase)
                current_role = None
                current_phrase = []

        # Add the last entity
        if current_role:
            combined_roles[current_role] = " ".join(current_phrase)

        # Store event representation
        event_representation.append({"Event": verb, "Roles": combined_roles})

    return event_representation

# Sample SRL output
output = {
    'verbs': [
        {'verb': 'blocked', 'description': '[ARG0: Protesters] [V: blocked] [ARG1: the main highway in Paris] [ARGM-TMP: last night] [ARGM-CAU: due to rising fuel prices] .',
         'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'B-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'I-ARGM-CAU', 'O']},
        {'verb': 'rising', 'description': 'Protesters blocked the main highway in Paris last night due to [V: rising] [ARG1: fuel prices] .',
         'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}
    ],
    'words': ['Protesters', 'blocked', 'the', 'main', 'highway', 'in', 'Paris', 'last', 'night', 'due', 'to', 'rising', 'fuel', 'prices', '.']
}

# Convert BIO tags to structured format
events = combine_bio_tags(output)

# Print structured event representation
for event in events:
    print(f"Event: {event['Event']}")
    for role, phrase in event["Roles"].items():
        print(f"  {role}: {phrase}")
    print()


Event: blocked
  ARG0: Protesters
  V: blocked
  ARG1: the main highway in Paris
  ARGM-TMP: last night
  ARGM-CAU: due to rising fuel prices

Event: rising
  V: rising
  ARG1: fuel prices



# With News Data

In [2]:
import pandas as pd

pd.set_option("max.colwidth",0)

In [3]:
df = pd.read_parquet("../data/ed_output.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 6458 to 1199
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           50 non-null     object
 1   Source          50 non-null     object
 2   Author          12 non-null     object
 3   Published Date  50 non-null     object
 4   URL             50 non-null     object
 5   Country         50 non-null     object
 6   Content         50 non-null     object
 7   ed_results      50 non-null     object
dtypes: object(8)
memory usage: 3.5+ KB


In [7]:
# print(df["Content"].iloc[0])
for data in df["Content"].iloc[0].split("\n\n")[:5]:
    output = predictor.predict(data)
    events = combine_bio_tags(output)

    for event in events:
        print(f"Event: {event['Event']}")
        for role, phrase in event["Roles"].items():
            print(f"  {role}: {phrase}")
        print()

Event: is
  ARGM-ADV: Apart from a divergence among different income groups
  ARG1: China ’s consumption trend
  ARGM-TMP: this year
  V: is
  ARG2: polarised among sectors

Event: polarised
  ARGM-ADV: Apart from a divergence among different income groups
  ARG1: China ’s consumption trend
  ARGM-TMP: this year
  V: polarised
  ARGM-LOC: among sectors
  ARGM-CAU: as service consumption such as concerts , tourism and catering have been better than the sale of goods – which has been more suppressed by worsening income prospects

Event: have
  V: have

Event: been
  ARG1: service consumption such as concerts , tourism and catering
  V: been
  ARG2: better than the sale of goods – which has been more suppressed by worsening income prospects

Event: has
  V: has

Event: been
  V: been

Event: suppressed
  ARG1: goods
  R-ARG1: which
  ARGM-EXT: more
  V: suppressed
  ARG0: by worsening income prospects

Event: worsening
  V: worsening
  ARG1: income prospects

Event: said
  ARG0: Zhang
  V

## With LLM

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

In [2]:
# Set up your OpenAI API key

# Initialize the LLM (GPT-4 or GPT-3.5)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",  # You can use "gpt-3.5-turbo" for faster results
    temperature=0.3  # Lower temp for more factual responses
)

def summarize_news(article):
    """
    Summarizes the news article into key events.
    """
    system_message = SystemMessage(
        content="You are an expert in news summarization. Extract key events from the given news article and summarize them concisely."
    )
    
    user_message = HumanMessage(
        content=f"""
        Summarize the following news article by identifying key events and their main details.
        Your summary should be concise, factual, and structured, covering the **who, what, where, when, and why** of major events.
        Focus only on the most **important** developments and avoid unnecessary details.
        
        News Article:
        {article}
        
        **Expected Output Format:**
        - [One-sentence summary]
        - [One-sentence summary]
        - [One-sentence summary]
        """
    )

    response = llm([system_message, user_message])
    return response.content


# Example news article
news_article = """
Apart from a divergence among different income groups, China’s consumption trend this year is polarised among sectors, 
as service consumption such as concerts, tourism and catering have been better than the sale of goods – which has been 
more suppressed by worsening income prospects, Zhang said.

Sandalwood data showed that the total booking value on two major Chinese travel platforms, Trip.com and Tongcheng Travel, 
hit a record high in July, thanks to the high demand during the summer holiday.

Despite the slightly weakening momentum in August, it still maintained double-digit growth: in the first half of August, 
bookings on Trip.com increased by 85 per cent by value, year on year, and on Tongcheng Travel they increased by 60 per cent.

The summer box office data also reached a new historical high this year. According to movie industry data provider Dengta, 
the box office in China from June to August reached 20.62 billion yuan (US$2.83 billion), compared with the previous record 
of 17.78 billion yuan achieved in 2019.

High-frequency data on the property sector, such as the volume of new home sales in major cities, has remained weak and 
shown little improvement, despite Beijing’s relaxing measures, economists said.
"""

# Get summarized events
summary = summarize_news(news_article)
print("\n **Summary of Events:**\n", summary)


/tmp/ipykernel_289377/2405720650.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/tmp/ipykernel_289377/2405720650.py:35: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([system_message, user_message])



 **Summary of Events:**
 - China's consumption trend in 2021 shows a divergence among income groups, with service consumption like concerts and tourism outperforming goods sales due to worsening income prospects.
- Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel hit a record high in July, driven by high summer holiday demand.
- Despite a slight weakening in August, summer box office data in China reached a new historical high of 20.62 billion yuan, surpassing the 2019 record, while the property sector's new home sales volume remains weak despite Beijing's measures.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
import json

# Define JSON schema for structured event extraction
response_schemas = [
    ResponseSchema(name="Event", description="The main action happening in the news."),
    ResponseSchema(name="Who", description="Who is involved in the event."),
    ResponseSchema(name="What", description="What happened."),
    ResponseSchema(name="Where", description="The location of the event."),
    ResponseSchema(name="When", description="The timeframe of the event."),
    ResponseSchema(name="Why", description="The reason or cause of the event."),
    ResponseSchema(name="Supporting Evidence", description="A relevant quote from the article."),
]

# Create the structured output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Define the prompt template
prompt_template = PromptTemplate(
    template="""Analyze the following news article and extract structured event representations. Identify major events along with the entities involved.
    
    **Return the output in valid JSON format**.

    {format_instructions}

    **News Article:**
    {article}
    """,
    input_variables=["article"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

def extract_structured_events(article):
    """
    Extracts structured events using OpenAI's function calling.
    """
    prompt = prompt_template.format(article=article)

    response = llm.invoke(prompt)
    
    response_text = response.content.strip()

    # Parse structured response
    try:
        structured_events = output_parser.parse(response_text)
    except Exception as e:
        structured_events = {"error": f"Failed to parse JSON output: {str(e)}", "raw_response": response_text}

    return structured_events

# Get structured event
entity_lst = []
for event in summary.split("\n"):
    structured_events = extract_structured_events(event)
    # Print structured events
    entity_lst.append(structured_events)
    print("Added.....")


Added.....
Added.....
Added.....


In [4]:
entity_lst

[{'Event': 'Consumption trend divergence among income groups',
  'Who': 'China',
  'What': 'shows a divergence with service consumption outperforming goods sales',
  'Where': 'China',
  'When': '2021',
  'Why': 'due to worsening income prospects',
  'Supporting Evidence': "China's consumption trend in 2021 shows a divergence among income groups, with service consumption like concerts and tourism outperforming goods sales due to worsening income prospects."},
 {'Event': 'Total booking value hit a record high',
  'Who': 'Chinese travel platforms Trip.com and Tongcheng Travel',
  'What': 'experienced a surge in bookings',
  'Where': 'on their platforms',
  'When': 'in July',
  'Why': 'driven by high summer holiday demand',
  'Supporting Evidence': 'Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel hit a record high in July, driven by high summer holiday demand.'},
 {'Event': 'Summer box office data in China reaches a new historical high',
  'Who': 'China',
  'W

In [12]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from allennlp.data.tokenizers import SpacyTokenizer

# Load the SRL model
predictor = Predictor.from_path(
    "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"
)

# Manually set tokenizer
predictor._tokenizer = SpacyTokenizer(language="en_core_web_sm", pos_tags=True)

summary = [
    "China's consumption trend in 2021 shows a divergence among income groups, with service consumption outperforming goods sales due to worsening income prospects.",
    "Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel hit a record high in July, driven by high demand during the summer holiday.",
    "Despite a slight weakening in August, summer box office data in China reached a new historical high of 20.62 billion yuan, surpassing the previous record set in 2019."
]
allennlp_event_lst = []
for sentence in summary:
    # Predict SRL roles
    output = predictor.predict(sentence)
    allennlp_event_lst.append(output)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
allennlp_event_lst

[{'verbs': [{'verb': 'shows',
    'description': "[ARG0: China 's consumption trend in 2021] [V: shows] [ARG1: a divergence among income groups] , [ARGM-PRD: with service consumption outperforming goods sales due to worsening income prospects] .",
    'tags': ['B-ARG0',
     'I-ARG0',
     'I-ARG0',
     'I-ARG0',
     'I-ARG0',
     'I-ARG0',
     'B-V',
     'B-ARG1',
     'I-ARG1',
     'I-ARG1',
     'I-ARG1',
     'I-ARG1',
     'O',
     'B-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'I-ARGM-PRD',
     'O']},
   {'verb': 'outperforming',
    'description': "China 's consumption trend in 2021 shows a divergence among income groups , with [ARG0: service consumption] [V: outperforming] [ARG1: goods sales] [ARGM-CAU: due to worsening income prospects] .",
    'tags': ['O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
     'O',
 

In [16]:
# Convert BIO tags to structured format
for event in allennlp_event_lst:
    events = combine_bio_tags(event)

    # Print structured event representation
    for event in events:
        print(f"Event: {event['Event']}")
        for role, phrase in event["Roles"].items():
            print(f"  {role}: {phrase}")
        print()

Event: shows
  ARG0: China 's consumption trend in 2021
  V: shows
  ARG1: a divergence among income groups
  ARGM-PRD: with service consumption outperforming goods sales due to worsening income prospects

Event: outperforming
  ARG0: service consumption
  V: outperforming
  ARG1: goods sales
  ARGM-CAU: due to worsening income prospects

Event: worsening
  V: worsening
  ARG1: income prospects

Event: hit
  ARG0: Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel
  V: hit
  ARG1: a record high
  ARGM-TMP: in July
  ARGM-PRD: driven by high demand during the summer holiday

Event: driven
  ARG1: Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel
  V: driven
  ARG0: by high demand
  ARGM-TMP: during the summer holiday

Event: reached
  ARGM-ADV: surpassing the previous record set in 2019
  ARG0: summer box office data in China
  V: reached
  ARG1: a new historical high of 20.62 billion yuan

Event: surpassing
  ARG0: summer box offic

## NER

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

summary = [
    "China's consumption trend in 2021 shows a divergence among income groups, with service consumption outperforming goods sales due to worsening income prospects.",
    "Total booking value on Chinese travel platforms Trip.com and Tongcheng Travel hit a record high in July, driven by high demand during the summer holiday.",
    "Despite a slight weakening in August, summer box office data in China reached a new historical high of 20.62 billion yuan, surpassing the previous record set in 2019."
]

ner_results = []
for data in summary:
    result = nlp(data)
    ner_results.append(result)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [5]:
ner_results

[[{'entity': 'B-LOC',
   'score': 0.9998479,
   'index': 1,
   'word': 'China',
   'start': 0,
   'end': 5}],
 [{'entity': 'B-MISC',
   'score': 0.99977785,
   'index': 5,
   'word': 'Chinese',
   'start': 23,
   'end': 30},
  {'entity': 'B-ORG',
   'score': 0.99913543,
   'index': 8,
   'word': 'Trip',
   'start': 48,
   'end': 52},
  {'entity': 'I-ORG',
   'score': 0.9943826,
   'index': 9,
   'word': '.',
   'start': 52,
   'end': 53},
  {'entity': 'I-ORG',
   'score': 0.9972887,
   'index': 10,
   'word': 'com',
   'start': 53,
   'end': 56},
  {'entity': 'B-ORG',
   'score': 0.9989667,
   'index': 12,
   'word': 'Tong',
   'start': 61,
   'end': 65},
  {'entity': 'I-ORG',
   'score': 0.9958977,
   'index': 13,
   'word': '##cheng',
   'start': 65,
   'end': 70},
  {'entity': 'I-ORG',
   'score': 0.99571085,
   'index': 14,
   'word': 'Travel',
   'start': 71,
   'end': 77}],
 [{'entity': 'B-LOC',
   'score': 0.9998381,
   'index': 13,
   'word': 'China',
   'start': 64,
   'end': 

In [7]:
def combine_bio_entities(ner_results):
    """
    Combines BIO-tagged named entity recognition (NER) results into complete entities.

    Parameters:
    - ner_results: List of lists containing token-level BIO-tagged entities.

    Returns:
    - List of combined entity names with labels.
    """
    combined_entities = []
    
    for sentence in ner_results:
        current_entity = None
        current_label = None
        current_words = []
        current_start = None
        current_end = None
        current_score = []
        
        for token in sentence:
            entity = token["entity"]
            word = token["word"]
            score = token["score"]
            start = token["start"]
            end = token["end"]

            # If it's a beginning token (B-), start a new entity
            if entity.startswith("B-"):
                # Store previous entity before starting a new one
                if current_entity:
                    combined_entities.append({
                        "entity": current_entity,
                        "label": current_label,
                        "score": sum(current_score) / len(current_score),  # Average score
                        "start": current_start,
                        "end": current_end
                    })

                # Start new entity
                current_entity = word
                current_label = entity[2:]  # Remove "B-" prefix
                current_words = [word]
                current_start = start
                current_end = end
                current_score = [score]

            # If it's an inside token (I-) and matches the current entity type, continue
            elif entity.startswith("I-") and current_label == entity[2:]:
                current_words.append(word)
                current_end = end  # Update end position
                current_score.append(score)

            # If it's a standalone entity or something went wrong, store the previous entity
            else:
                if current_entity:
                    combined_entities.append({
                        "entity": " ".join(current_words).replace(" ##", ""),  # Remove subword markers
                        "label": current_label,
                        "score": sum(current_score) / len(current_score),
                        "start": current_start,
                        "end": current_end
                    })
                current_entity = None
                current_label = None
                current_words = []
                current_start = None
                current_end = None
                current_score = []

        # Store the last entity if any
        if current_entity:
            combined_entities.append({
                "entity": " ".join(current_words).replace(" ##", ""),  
                "label": current_label,
                "score": sum(current_score) / len(current_score),
                "start": current_start,
                "end": current_end
            })

    return combined_entities


# Run the function
combined_entities = combine_bio_entities(ner_results)


In [8]:
combined_entities

[{'entity': 'China',
  'label': 'LOC',
  'score': 0.9998478889465332,
  'start': 0,
  'end': 5},
 {'entity': 'Chinese',
  'label': 'MISC',
  'score': 0.9997778534889221,
  'start': 23,
  'end': 30},
 {'entity': 'Trip',
  'label': 'ORG',
  'score': 0.9969355861345927,
  'start': 48,
  'end': 56},
 {'entity': 'Tongcheng Travel',
  'label': 'ORG',
  'score': 0.9968584179878235,
  'start': 61,
  'end': 77},
 {'entity': 'China',
  'label': 'LOC',
  'score': 0.99983811378479,
  'start': 64,
  'end': 69}]

In [9]:
ner_results

[[{'entity': 'B-LOC',
   'score': 0.9998479,
   'index': 1,
   'word': 'China',
   'start': 0,
   'end': 5}],
 [{'entity': 'B-MISC',
   'score': 0.99977785,
   'index': 5,
   'word': 'Chinese',
   'start': 23,
   'end': 30},
  {'entity': 'B-ORG',
   'score': 0.99913543,
   'index': 8,
   'word': 'Trip',
   'start': 48,
   'end': 52},
  {'entity': 'I-ORG',
   'score': 0.9943826,
   'index': 9,
   'word': '.',
   'start': 52,
   'end': 53},
  {'entity': 'I-ORG',
   'score': 0.9972887,
   'index': 10,
   'word': 'com',
   'start': 53,
   'end': 56},
  {'entity': 'B-ORG',
   'score': 0.9989667,
   'index': 12,
   'word': 'Tong',
   'start': 61,
   'end': 65},
  {'entity': 'I-ORG',
   'score': 0.9958977,
   'index': 13,
   'word': '##cheng',
   'start': 65,
   'end': 70},
  {'entity': 'I-ORG',
   'score': 0.99571085,
   'index': 14,
   'word': 'Travel',
   'start': 71,
   'end': 77}],
 [{'entity': 'B-LOC',
   'score': 0.9998381,
   'index': 13,
   'word': 'China',
   'start': 64,
   'end': 